In [1]:
# pip install nir --quiet
# pip install fxpmath --quiet
# pip install bitstring --quiet

In [ ]:
import numpy as np
import itertools
from fxpmath import Fxp

import nir
import bitstring as bs
from bitstring import BitArray
from bitstring import BitStream

import os    
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
# Customize your bitstream!
max_num_neurons_per_layer = 256 # Specify the max number of neurons for each layer
fixed_point_representation = [16, 7] # [num of bits, demical granularity]

In [3]:
# import graphs from NIR format
victim = nir.read("test.nir")
output_file_name = 'test'

In [4]:
# Inbuilt functions for the conversion to the .nerve file format
'''
Inbuilt functions automatically append lists to be the of the max layer size. This behavior is intentional and compliant
with .nerve file formatting

The entire notebook assumes that you follow the below constraints:
- Your network is quantized
- Your network uses exclusively LIF neurons
- Your hardware assumes that all neurons MUST be programmed
- All layers are fully connected and follow the naming convention "fc#" where # is the position of the layer in the network
- You have no life
'''
# only supports 1 dimensional matrices
def tilt_weights(weight_matrix):
    tilted_weights = [0 for i in range(len(weight_matrix))]  
    for i in range(len(weight_matrix)):
        if weight_matrix[i] > 0:
            tilted_weights[i] = 1
    tilted_weights += (max_num_neurons_per_layer - len(weight_matrix)) * [0]
    return tilted_weights

# can't figure out how to do this inteligently
def tilt_weights_2d(weight_matrix):
    tilted_weights = [[] for i in range(max_num_neurons_per_layer)]
    filler = [0 for i in range(max_num_neurons_per_layer)]
    for j in range(len(weight_matrix)):
        tilted_weights[j] = tilt_weights(weight_matrix[j])
    for i in range(len(weight_matrix), max_num_neurons_per_layer):
        tilted_weights[i] = filler
    return tilted_weights

# does not support 2d matrices
def FPR_biases_transform(biases):
    FPR_bias_matrix = [None for i in range(max_num_neurons_per_layer)]
    filler = [0 for i in range(fixed_point_representation[0])]
    for i in range(len(biases)):
        x = Fxp(biases[i] , True , fixed_point_representation[0] , fixed_point_representation[1] )
        bunga = [None for k in range(fixed_point_representation[0])]
        for j in range(fixed_point_representation[0]):
            bunga[j] = int(x.bin()[j])
        FPR_bias_matrix[i] = bunga
    for i in range(len(biases), max_num_neurons_per_layer):
        FPR_bias_matrix[i] = filler
    return FPR_bias_matrix

# is mega-tarded
def fc_conn_matrix(node_count):
    CM = [1 for i in range(node_count)]
    CM += (max_num_neurons_per_layer - node_count) * [0]
    return CM

def create_fc_conn_matrix(prev_node_count, cur_node_count):
    CM = [None for i in range (max_num_neurons_per_layer)]
    for i in range(cur_node_count):
        CM[i] = fc_conn_matrix(prev_node_count)
    for i in range(cur_node_count, max_num_neurons_per_layer):
        CM[i] = fc_conn_matrix(0)
    return CM

def create_zeroth_layer_conn_matrix(node_count):
    CM = [[0 for j in range(node_count)] for i in range(node_count)]
    for i in range(node_count):
        CM[i][i] = 1
    return CM

def create_zeroth_layer_weight_matrix(node_count):
    WM = [[0 for j in range(node_count)] for i in range(node_count)]
    for i in range(node_count):
        WM[i][i] = 1
    return WM

def create_zeroth_layer_bias_matrix(node_count):
    BM = [[0 for j in range(fixed_point_representation[0])] for i in range(node_count)]
    return BM

def create_zeroth_layer(node_count):
    ZL = [create_zeroth_layer_bias_matrix(node_count), create_zeroth_layer_weight_matrix(node_count), create_zeroth_layer_weight_matrix(node_count)]
    return ZL

def decrypt_fc_nodes(fcn):
    node_outputs = fcn.output_type["output"][0] # This trick works for ONLY FULLY CONNECTED NETWORKS
    node_inputs = fcn.input_type["input"][0] # This trick works for ONLY FULLY CONNECTED NETWORKS
    weight_matrix = fcn.weight
    biases = fcn.bias
    layer_info = [FPR_biases_transform(biases), tilt_weights_2d(weight_matrix), create_fc_conn_matrix(node_inputs, node_outputs)]
    return layer_info

def ascii_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

def format_binary_file(input_file, output_file):
    with open(input_file, 'rb') as f:
        # Read the raw binary data
        data = f.read()

    # Convert to a binary string representation
    binary_string = ''.join(format(byte, '08b') for byte in data)

    # Prepare to collect 32-bit lines
    lines = []
    for i in range(0, len(binary_string), 32):
        # Get the next 16 bits
        chunk = binary_string[i:i+32]
        lines.append(chunk)

    # Write to the output file with line endings
    with open(output_file, 'w') as f:
        for line in lines:
            f.write(line + '\n')

In [5]:
layers_node_input_count = victim.nodes["input"].input_type.get("input") # formatted in last layer -> first layer
num_of_layers = 1 # will allow us to index into the first layer and create 

for i in victim.nodes:
    if(str(i).startswith("fc")):
        if(num_of_layers == 1):
            input_layer_size = victim.nodes[i].input_type["input"][0]
        num_of_layers += 1

'''
Information should be stored as the following for each neuron n
[Bias[n], weights[n], connectivity[n]]

layer_info is a 3d array that is structed as the following
[x][y][z] -> x is the number of layers, y is 3 as each nueron has three values associated with it, z is the number of nodes

The final bitstream should have a readable header, a body that contains every node's info sequentially in 528 bit chunks, and a readable footer

Header contains the following information:
[number of layers, max number of neurons per layer, footer length]

all values are 16 bits

Footer contains only copyright information
'''

layer_cnt = 0
layer_info = [None for i in range(num_of_layers)]
      
# The zeroth layer of the network must be created from information provided in the non fc nodes
zeroth_layer = create_zeroth_layer(input_layer_size)
layer_info[0] = zeroth_layer
# all subsequent layers can be constructed from the NIR format 
for i in victim.nodes:
    if(str(i).startswith("fc")):
        layer_cnt += 1
        layer_name = "fc" + str(layer_cnt)
        layer_info[layer_cnt] = decrypt_fc_nodes(victim.nodes[layer_name])

NERVE_body = bs.BitArray()

for l in range(num_of_layers):
    for n in range(max_num_neurons_per_layer):
        funny_haha_append = "0x" + format(n, '04X')
        NERVE_body.append(funny_haha_append)
        for i in range(3):
            NERVE_body.append(BitArray(layer_info[l][i][n]))
                    


# Header
hex_length = format(num_of_layers, '04X')
convert_length = "0x" + hex_length
hex_node = format(max_num_neurons_per_layer, '04X')
convert_node = "0x" + hex_node
header_layer_count = BitArray(convert_length)
header_node_count = BitArray(convert_node)
convert_bias_count = "0x" + format(fixed_point_representation[0], '04X')
header_bias_count = BitArray(convert_bias_count)
convert_bias_dec = "0x" + format(fixed_point_representation[1], '04X')
header_bias_dec = BitArray(convert_bias_dec)
# Copyright 
copyright_statement = "Copyright (c) 2024 Vedang Verma, Eitan Tse, Prarthik Sathyanarayanan, Piysuh Pahuja, Rahim Khan"
bin_copyright = ascii_to_binary(copyright_statement)
bit_copyright = [None for k in range(len(bin_copyright))]
for j in range(len(bin_copyright)):
    bit_copyright[j] = int(bin_copyright[j])
len_of_footer = len(bit_copyright)//8
convert_footer_len = "0x" + format(len_of_footer, '04X')

NERVE_header = BitArray()
NERVE_header.append(header_layer_count)
NERVE_header.append(header_node_count)
NERVE_header.append(header_bias_count)
NERVE_header.append(convert_footer_len)

NERVE_footer = BitArray(bit_copyright)

In [6]:
NERVE_file = BitStream()
NERVE_file.append(NERVE_header)
NERVE_file.append(NERVE_body)
NERVE_file.append(NERVE_footer)

raw_filename = output_file_name + '_raw.nerve'
f = open(raw_filename, 'wb')
NERVE_file.tofile(f)

filename = output_file_name + '.nerve'

format_binary_file(raw_filename, filename)
